## LCEL 인터페이스

사용자 정의 체인을 가능한 쉽게 만들 수 있도록, [`Runnable`](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable) 프로토콜을 구현했습니다. 

`Runnable` 프로토콜은 대부분의 컴포넌트에 구현되어 있습니다.

이는 표준 인터페이스로, 사용자 정의 체인을 정의하고 표준 방식으로 호출하는 것을 쉽게 만든다. 표준 인터페이스에는 다음이 포함된다.

- [`stream`](#stream): 응답의 청크를 스트리밍합니다.
- [`invoke`](#invoke): 입력에 대해 체인을 호출합니다.
- [`batch`](#batch): 입력 목록에 대해 체인을 호출합니다.

비동기 메소드도 있습니다.

- [`astream`](#async-stream): 비동기적으로 응답의 청크를 스트리밍합니다.
- [`ainvoke`](#async-invoke): 비동기적으로 입력에 대해 체인을 호출합니다.
- [`abatch`](#async-batch): 비동기적으로 입력 목록에 대해 체인을 호출합니다.
- [`astream_log`](#async-stream-intermediate-steps): 최종 응답뿐만 아니라 발생하는 중간 단계를 스트리밍합니다.

In [5]:
# API KEY를 환경변수로 관리하기 위한 설정파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [6]:
# LangSmith 추적을 설정
from langchain_teddynote import logging

# 프로젝트 이름을 입력
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


LCEL 문법을 사용하여 chain을 생성

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ChatOpenAI 모델을 인스턴스화한다
model = ChatOpenAI()
# 주어진 토픽에 대한 농담을 요청하는 프롬프트 템플릿을 생성한다
prompt = PromptTemplate.from_template("{topic} 에 대하여 3문장으로 설명해줘.")
# 프롬프트와 모델을 연결하여 대화 체인을 생성한다
chain = prompt | model | StrOutputParser()

In [9]:
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='{topic} 에 대하여 3문장으로 설명해줘.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x112aaf290>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x112fa8290>, root_client=<openai.OpenAI object at 0x112a680d0>, root_async_client=<openai.AsyncOpenAI object at 0x112f93d10>, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

### stream: 실시간 출력

- 이 함수는 `chain.stream`메서드를 사용하여 주어진 토픽에 대한 데이터 스트림을 생성하고, 이 스트림을 반복하여 각 데이터의 내용(content)을 즉시 출력한다. 
- `end=""`인자는 출력 후 줄바꿈을 하지 않도록 설정.
- `flush=True`인자는 출력버퍼를 즉시 비우도록 한다.

In [11]:
# chain.stream 메서드를 사용하여 '멀티모달' 토픽에 대한 스트림을 생성하고 반복한다
for token in chain.stream({"topic": "멀티모달"}):
    # 스트림에서 받은 데이터의 내용을 출력한다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다
    print(token, end="", flush=True)

멀티모달은 여러 가지 형태의 커뮤니케이션 매체를 병행하여 정보를 전달하는 방식을 의미합니다. 이는 텍스트, 이미지, 음성, 영상 등 다양한 형태의 미디어를 이용해 사용자에게 효과적으로 정보를 전달할 수 있게 도와줍니다. 멀티모달은 사용자 경험을 향상시키고 정보 전달의 효율을 증대시키는 데 도움을 줄 수 있습니다.

### invoke: 호출
`chain` 객체의 `invoke` 메서드는 주제를 인자로 받아 해당 주제에 대한 처리를 수행한다.

In [12]:
# chain 객체에 invoke 메서드를 호출하고 'ChatGPT'라는 주제로 딕셔너리를 전달한다.
chain.invoke({"topic": "ChatGPT"})

'ChatGPT 는 자연어 생성 모델로, 사용자와 대화를 통해 다양한 주제에 대한 정보를 제공합니다. 인공지능 기술을 활용하여 자연스러운 대화를 제공하며, 실제 사람과 헷갈리는 수준의 자연어 이해 능력을 갖추고 있습니다. ChatGPT 를 통해 사용자는 편리하고 신속하게 원하는 정보를 얻을 수 있습니다.'

### batch: 배치(단위 실행)
함수 `chain.batch`는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 `topic`키의 값을 사용하여 일괄 처리를 수행한다

In [13]:
# 주어진 토픽 리스트를 batch 처리하는 함수 호출
answer = chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])

In [14]:
answer[0]

'ChatGPT는 OpenAI에서 개발된 대화형 인공지능 모델로, 사용자와 자연스럽게 대화를 주고받을 수 있습니다. 다양한 주제에 대한 대화를 지원하며, 챗봇이나 가상 비서 등 다양한 용도로 활용됩니다. 기술적으로 발전하면서 점차 더 자연스러운 대화를 이끌어내는 역할을 하고 있습니다.'

In [15]:
answer[1]

'Instagram은 사진과 동영상을 공유할 수 있는 소셜 미디어 플랫폼으로, 사용자들은 자신의 일상을 공유하고 소통할 수 있습니다. 또한 다양한 필터와 편집 기능을 제공하여 보다 멋진 사진을 만들 수 있습니다. 해시태그를 통해 관심사나 트렌드를 탐색하고 팔로워들과 상호작용할 수 있습니다.'

`max_concurrency` 매개변수를 사용하여 동시 요청 수를 설정할 수 있다. 

`config` 딕셔너리는 `max_concurrency`키를 통해 동시에 처리할 수 있는 최대 작업 수를 설정한다. 여기서는 최대 3개의 작업을 동시에 처리하도록 설정되어 있다. 

In [18]:
chain.batch(
    [
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "멀티모달"},
        {"topic": "프로그래밍"},
        {"topic": "머신러닝"},
    ],
    config={"max_concurrency": 3},
)

['ChatGPT는 인공지능 기반의 챗봇 서비스로서 자연어 이해 및 대화 기능을 제공합니다. 사용자의 질문에 답변하거나 대화를 이어가는 데 활용되며, 다양한 주제에 대해 지식을 제공하고 이해할 수 있습니다. ChatGPT는 머신러닝 알고리즘을 기반으로 작동하여 사용자 경험을 향상시키고 효율적인 의사소통을 도와줍니다.',
 'Instagram은 SNS 앱으로, 사용자들이 이미지와 비디오를 공유하고 친구들과 소통하며 콘텐츠를 발견할 수 있는 플랫폼입니다. 사용자들은 팔로우, 좋아요, 코멘트 등을 통해 다양한 활동을 할 수 있고, 브랜드나 인플루언서들은 마케팅을 위해 활발히 활용하고 있습니다. 꾸준히 성장하는 인기를 지닌 Instagram은 전 세계에서 많은 이용자들에게 사랑받고 있는 소셜미디어 플랫폼 중 하나입니다.',
 '멀티모달이란 여러 종류의 다양한 미디어를 결합하여 정보를 전달하거나 상호작용하는 방식을 말합니다. 예를 들어 텍스트, 이미지, 동영상 및 음성 등 다양한 형태의 미디어를 활용하여 사용자에게 정보를 제공하는 것을 말합니다. 멀티모달은 사용자들에게 더 풍부한 경험과 이해를 제공하여 소통과 상호작용의 효율성을 높여줍니다.',
 '프로그래밍이란 컴퓨터에게 명령과 로직을 전달하여 원하는 작업을 수행하도록 하는 작업을 말합니다. 프로그래밍을 통해 컴퓨터를 제어하고 다양한 소프트웨어를 개발할 수 있습니다. 프로그래밍은 문제 해결능력을 향상시키고 창의적인 사고를 키우는데 도움이 됩니다.',
 '머신러닝은 컴퓨터가 데이터에서 학습하고 패턴을 발견하여 예측하거나 의사결정을 하는 인공지능 기술이다. 이를 위해 데이터를 통해 모델을 학습하고, 새로운 데이터를 분석하여 결과를 예측한다. 머신러닝은 다양한 분야에서 활용되며, 빅데이터와 함께 혁신적인 기술을 이끌어내고 있다.']

### async stream: 비동기 스트림
함수 `chain.astream`은 비동기 스트림을 생성하며, 주어진 토픽에 대한 메시지를 비동기적으로 처리한다.

- 비동기 for 루프(`async for`)를 사용하여 스트림에서 메시지를 순차적으로 받아오고, `print`함수를 통해 메시지의 내용(`s.content`)을 즉시 출력한다. 
- `end=""`는 출력 후 줄바꿈을 하지 않도록 설정
- `flush=True`는 출력 버퍼를 강제로 비워 즉시 출력되도록 한다.

In [19]:
# 비동기 스트림을 사용하여 'YouTube' 토픽의 메시지를 처리한다
async for token in chain.astream({"topic": "YouTube"}):
    # 메시지 내용을 출력한다. 줄바꿈 없이 바로 출력하고 버퍼를 비웁니다.
    print(token, end="", flush=True)

YouTube는 동영상을 업로드하고 시청할 수 있는 온라인 동영상 플랫폼이다. 다양한 주제와 콘텐츠를 제공하여 사용자들이 자신에게 맞는 영상을 쉽게 찾아 시청할 수 있다. 또한, 유명한 크리에이터들이 활동하며 시청자와의 소통을 통해 광범위한 팬층을 형성하고 있다.

### async invoke: 비동기 호출

`chain` 객체의 `ainvoke` 메서드는 비동기적으로 주어진 인자를 사용하여 작업을 수행한다. 여기서 `topic`이라는 키와 `NVDA`(엔비디아의 티커)라는 값을 가진 딕셔너리를 인자로 전달하고 있다. 메서드는 특정 토픽에 대한 처리를 비동기적으로 요청하는 데 사용될 수 있다.

In [20]:
# 비동기 체인 객체의 'ainvoke' 메서드를 호출하여 'NVDA' 토픽을 처리한다
my_process = chain.ainvoke({"topic": "NVDA"})

In [21]:
# 비동기로 처리되는 프로세스가 완료될 때까지 기다린다.
await my_process

'NVDA는 엔비디아 코퍼레이션의 주식을 나타내는 주식 코드이다. 엔비디아는 그래픽 카드와 데이터 처리 솔루션을 생산하는 글로벌 기술 기업으로, 주로 게임 산업과 인공지능 분야에서 활약하고 있다. NVDA 주식은 기술 시장에서 선두적인 위치를 차지하고 있으며 투자자들 사이에서도 인기가 높다.'

### async batch: 비동기 배치
함수 `abatch`는 비동기적으로 일련의 작업을 일괄 처리한다.

이 예시에서는 `chain`객체의 `abatch` 메서드를 사용하여 `topic` 에 대한 작업을 비동기적으로 처리하고 있다.

`await` 키워드는 해당 비동기 작업이 완료될 때까지 기다리는 데 사용된다.

In [22]:
# 주어진 토픽에 대해 비동기적으로 일괄 처리를 수행한다
my_abatch_process = chain.abatch(
    [{"topic": "YouTube"}, {"topic": "instagram"}, {"topic": "Facebook"}]
)

In [23]:
# 비동기로 처리되는 일괄 처리 프로세스가 완료될 때까지 기다린다.
await my_abatch_process

['YouTube는 비디오 공유 플랫폼으로, 사용자들이 자신의 동영상을 업로드하고 시청할 수 있는 서비스이다. 다양한 주제의 동영상 콘텐츠를 제공하며, 시청자들은 댓글을 남기거나 구독을 통해 채널을 팔로우할 수 있다. 크리에이터들은 광고 수익을 얻을 수 있으며, 사용자들은 무료로 다양한 콘텐츠를 즐길 수 있다.',
 'Instagram은 사진과 동영상을 공유하는 소셜 미디어 플랫폼으로, 사용자들은 콘텐츠를 업로드하고 좋아요, 댓글 등의 상호작용을 할 수 있습니다. 해시태그를 통해 쉽게 관심사나 주제별로 게시물을 찾을 수 있으며, 스토리 기능을 통해 일상을 재미있게 공유할 수 있습니다. 또한 인플루언서들을 통해 광고나 마케팅 활동을 할 수 있어 인기 있는 플랫폼입니다.',
 'Facebook은 전 세계적으로 가장 대중화된 소셜 네트워크 서비스 중 하나로, 사용자들이 친구들과 소통하고 콘텐츠를 공유할 수 있는 플랫폼이다. 사생활 보호 문제와 논란을 일으키기도 하지만, 사용자들은 여전히 다양한 정보를 얻고 소셜 미디어로 상호작용하는데 활용하고 있다. 또한 광범위한 광고 시스템과 데이터 수집 논란에도 불구하고 전 세계적으로 막대한 영향력을 행사하고 있다.']

### Parallel: 병렬성
LangChain Expression Language가 병렬 요청을 지원하는 방법을 살펴본다. 예를 들어, `RunnableParallel`을 사용할 때, 각 요소를 병렬로 실행한다

`langchain_core.runnables` 모듈의 `RunnableParalle` 클래스를 사용하여 두 가지 작업을 병렬로 실행하는 예시를 보여준다.

`ChatPromptTemplate.from_template` 메서드를 사용하여 주어진 `country`에 대한 **수도**와 **면적**을 구하는 2개의 체인(`chain1`, `chain2`)을 만든다.

이 체인들은 각각 `model`과 파이프(`|`) 연산자를 통해 연결된다. 마지막으로, `RunnableParallel` 클래스를  사용하여 이 두 체인을 `capital`와 `area`이라는 키로 결합하여 동시에 실행할 수 있는 `combined` 객체를 생성한다.

In [24]:
from langchain_core.runnables import RunnableParallel

# {country} 이 수도를 물어보는 체인을 생성한다
chain1 = (
    PromptTemplate.from_template("{country} 의 수도는 어디야?")
    | model
    | StrOutputParser()
)

# { country} 의 면적을 물어보는 체인을 생성한다
chain2 = (
    PromptTemplate.from_template("{country} 의 면적은 얼마야?")
    | model
    | StrOutputParser()
)

# 위의 2개 체인을 동시에 생성하는 병렬 실행 체인을 생성한다
combined = RunnableParallel(capital=chain1, area=chain2)

`chain1.invoke()` 함수는 `chain1` 객체의 `invoke` 메시드를 호출한다

이때, `country`이라는 키에 `대한민국` 라는 값을 가진 딕셔너리를 인자로 전달한다

In [25]:
# chain1 를 실행한다
chain1.invoke({"country": "대한민국"})

'대한민국의 수도는 서울입니다.'

이번에는 `chain2.invoke()` 를 호출한다. `country` 키에 다른 국가인 `미국`을 전달한다

In [26]:
# chain2 를 실행한다
chain2.invoke({"country": "미국"})

'미국의 면적은 대략 9,833,520 제곱 킬로미터입니다.'

`combined` 객체의 `invoke` 메서드는 주어진 `country`에 대한 처리를 수행한다

이 예제에서는 `대한민국` 라는 주제를 `invoke` 메서드에 전달하여 실행한다.

In [27]:
# 병렬 실행 체인을 실행한다
combined.invoke({"country": "대한민국"})

{'capital': '대한민국의 수도는 서울이야.', 'area': '대한민국의 면적은 약 100,408km² 입니다.'}

### 배치에서의 병렬 처리

병렬 처리는 다른 실행 가능한 코드와 결합될 수 있다. 배치와 병렬 처리를 사용해 보도록 합시다.

`chain1.batch` 함수는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 "topic" 키에 해당하는 값을 처리한다. 이 예시에서는 "대한민국"와 "미국"라는 두 개의 토픽을 배치 처리하고 있다.

In [28]:
# 배치 처리를 수행한다
chain1.batch([{"country": "대한민국"}, {"country": "미국"}])

['대한민국의 수도는 서울이다.', '미국의 수도는 워싱턴 D.C.입니다.']

`chain2.batch` 함수는 여러 개의 딕셔너리를 리스트 형태로 받아, 일괄 처리(batch)를 수행한다.

이 예시에서는 `대한민국`과 `미국`이라는 두 가지 국가에 대한 처리를 요청한다.

In [29]:
# 배치 처리를 수행한다
chain2.batch([{"country": "대한민국"}, {"country": "미국"}])

['대한민국의 총 면적은 약 100,363 제곱 킬로미터입니다.', '미국의 면적은 약 9,833,520 km²입니다.']

`combined.batch` 함수는 주어진 데이터를 배치로 처리하는데 사용된다. 이 예시에서는 두 개의 딕셔너리 객체를 포함하는 리스트를 인자로 받아 각각 `대한민국`과 `미국` 두 나라에 대한 데이터를 배치 처리한다.

In [30]:
# 주어진 데이터를 배치로 처리한다
combined.batch([{"country": "대한민국"}, {"country": "미국"}])

[{'capital': '대한민국의 수도는 서울이다.', 'area': '대한민국의 면적은 약 100,363제곱킬로미터 입니다.'},
 {'capital': '미국의 수도는 워싱턴 D.C.입니다.', 'area': '미국의 총 면적은 약 9,833,520 km² 입니다.'}]